In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## Simple NN. You can change this if you want. If you change it, mention the architectural details in your report.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 50)
        self.out = nn.Linear(50, 10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.out(x) # logits out!
        return x

class Normalize(nn.Module):
    def forward(self, x):
        return (x - 0.1307)/0.3081

# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()

100%|██████████| 9.91M/9.91M [00:00<00:00, 41.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.11MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 11.6MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 3.72MB/s]


Sequential(
  (0): Normalize()
  (1): Net(
    (fc1): Linear(in_features=784, out_features=50, bias=True)
    (fc2): Linear(in_features=50, out_features=50, bias=True)
    (fc3): Linear(in_features=50, out_features=50, bias=True)
    (out): Linear(in_features=50, out_features=10, bias=True)
  )
)

In [2]:
def pgd_linf_untargeted(model, x, labels, k, eps, eps_step):
    # model.eval()
    ce_loss = torch.nn.CrossEntropyLoss()
    adv_x = x.clone().detach()
    adv_x.requires_grad_(True) 
    for _ in range(k):
        adv_x.requires_grad_(True)
        model.zero_grad()
        output = model(adv_x)
        # TODO: Calculate the loss
        loss = ce_loss(output, labels)
        loss.backward()
        # TODO: compute the adv_x
        # find delta, clamp with eps          
        delta = torch.clamp(adv_x + eps_step * adv_x.grad.sign() - x.data, min=-eps, max=eps)
        adv_x = torch.clamp(x.data + delta, min=0, max=1).detach_()
   
    return adv_x

def pgd_l2_untargeted(model, x, labels, k, eps, eps_step):
    # model.eval()
    ce_loss = torch.nn.CrossEntropyLoss()
    adv_x = x.clone().detach()
    adv_x.requires_grad_(True) 
    for _ in range(k):
        adv_x.requires_grad_(True)
        model.zero_grad()
        output = model(adv_x)
        batch_size = x.size()[0]
        # TODO: Calculate the loss
        loss = ce_loss(output, labels)
        loss.backward()
        grad = adv_x.grad.data
        # TODO: compute the adv_x
        # find delta, clamp with eps, project delta to the l2 ball
        # HINT: https://github.com/Harry24k/adversarial-attacks-pytorch/blob/master/torchattacks/attacks/pgdl2.py
        eps_for_division = 1e-10
        grad_norms = (
            torch.norm(grad.view(batch_size, -1), p=2, dim=1)
            + eps_for_division
        )  # nopep8
        grad = grad / grad_norms.view(batch_size, 1, 1, 1)
        adv_x = adv_x.detach() + eps_step * grad
        
        delta = adv_x  - x.data
        delta_norms = torch.norm(delta.view(batch_size, -1), p=2, dim=1)
        factor = eps / delta_norms
        factor = torch.min(factor, torch.ones_like(delta_norms))
        delta = delta * factor.view(-1, 1, 1, 1)
        
        adv_x = torch.clamp(x.data + delta, min=0, max=1).detach_()
   
    return adv_x

In [8]:
def test_model(model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        # tbar = tqdm(total=len(test_loader), desc="items of test:")
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        #     tbar.update(1)
        # tbar.close()
        print(f'Accuracy on images: {100 * correct / total}')

def test_model_robust(model, attack='pgd', eps=8/255, k=10):
    model.eval()
    # with torch.no_grad():abs
    correct = 0
    total = 0
    # tbar = tqdm(total=len(test_loader), desc="items of test:")
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        if attack == 'fgsm':
            adv_images = pgd_linf_untargeted(model, images, labels, k=1, eps=eps, eps_step=eps)
        else:  # pgd
            adv_images = pgd_linf_untargeted(model, images, labels, k=k, eps=eps, eps_step=eps/4)
        outputs = model(adv_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    #     tbar.update(1)
    # tbar.close()
    print(f'Accuracy on images: {100 * correct / total}')



In [5]:
from tqdm import tqdm

def train_model_ibp(model, num_epochs, eps_target=0.1, k_start=1.0, k_end=0.5):
    criterion = nn.CrossEntropyLoss()
    # optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    for epoch in tqdm(range(num_epochs)):
        model.train()
        running_loss = 0.0

        # gradual schedules for k and eps
        k = k_start + (k_end - k_start) * (epoch / max(num_epochs - 1, 1))
        eps_train = eps_target * (epoch / max(num_epochs - 1, 1))

        # tbar = tqdm(total=len(train_loader), desc="items of one epoch:")
        for i, data in enumerate(train_loader, 0):
            print(f"{i}/{len(train_loader)}", end="\r")
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            # forward
            outputs = model(images)
            l = torch.clamp(images - eps_train, 0.0, 1.0)
            u = torch.clamp(images + eps_train, 0.0, 1.0)
            l = (l - 0.1307) / 0.3081
            u = (u - 0.1307) / 0.3081
            B = images.size(0)
            l = l.view(B, -1)
            u = u.view(B, -1)
            net = model[1]

            # fc1 relu
            W, b = net.fc1.weight, net.fc1.bias
            Wp, Wn = torch.clamp(W, min=0), torch.clamp(W, max=0)
            lo = l @ Wp.t() + u @ Wn.t() + b
            hi = u @ Wp.t() + l @ Wn.t() + b
            l, u = F.relu(lo), F.relu(hi)

            # fc2 relu
            W, b = net.fc2.weight, net.fc2.bias
            Wp, Wn = torch.clamp(W, min=0), torch.clamp(W, max=0)
            lo = l @ Wp.t() + u @ Wn.t() + b
            hi = u @ Wp.t() + l @ Wn.t() + b
            l, u = F.relu(lo), F.relu(hi)

            # fc3 relu
            W, b = net.fc3.weight, net.fc3.bias
            Wp, Wn = torch.clamp(W, min=0), torch.clamp(W, max=0)
            lo = l @ Wp.t() + u @ Wn.t() + b
            hi = u @ Wp.t() + l @ Wn.t() + b
            l, u = F.relu(lo), F.relu(hi)

            # out layer (logits)
            W, b = net.out.weight, net.out.bias
            Wp, Wn = torch.clamp(W, min=0), torch.clamp(W, max=0)
            lo = l @ Wp.t() + u @ Wn.t() + b
            hi = u @ Wp.t() + l @ Wn.t() + b
            l, u = lo, hi

            # pessimistic logits
            logits_hat = u.clone()
            idx = torch.arange(labels.size(0), device=labels.device)
            logits_hat[idx, labels] = l[idx, labels]
            loss = k * criterion(outputs, labels) + (1.0 - k) * criterion(logits_hat, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        #     tbar.update(1)
        # tbar.close()
        scheduler.step()
        print(f'Epoch {epoch+1}/{num_epochs}, k={k:.3f}, eps_train={eps_train:.3f}, Loss: {running_loss/len(train_loader):.3f}')


In [6]:
model = nn.Sequential(Normalize(), Net()).to(device)
train_model_ibp(model, num_epochs=5, eps_target=0.1, k_start=1.0, k_end=0.5)

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [04:15<38:16, 255.19s/it]

Epoch 1/10, k=1.000, eps_train=0.000, Loss: 0.452


 20%|██        | 2/10 [08:35<34:26, 258.37s/it]

Epoch 2/10, k=0.944, eps_train=0.011, Loss: 0.269


...

Epoch 5/10, k=0.778, eps_train=0.044, Loss: 0.293


 60%|██████    | 6/10 [26:51<18:34, 278.69s/it]

Epoch 6/10, k=0.722, eps_train=0.056, Loss: 0.336


 60%|██████    | 6/10 [27:06<18:04, 271.16s/it]


KeyboardInterrupt: 

In [9]:

epsilon_values = [0, 1/255, 2/255, 4/255, 8/255, 16/255, 32/255]


print("Testing adversarily trained model acuracy:")
test_model(model)

for eps in epsilon_values:
    print(f"Testing adversarily trained model robustness, eps {eps}:")
    test_model_robust(model, attack='pgd', eps=eps, k=10)

for eps in epsilon_values:
    print(f"Testing adversarily FGSM, eps {eps}:")
    test_model_robust(model, attack='pgd', eps=eps, k=1)

Testing adversarily trained model acuracy:
Accuracy on images: 95.87
Testing adversarily trained model robustness, eps 0:
Accuracy on images: 95.87
Testing adversarily trained model robustness, eps 0.00392156862745098:
Accuracy on images: 95.54
Testing adversarily trained model robustness, eps 0.00784313725490196:
Accuracy on images: 95.26
Testing adversarily trained model robustness, eps 0.01568627450980392:
Accuracy on images: 94.45
Testing adversarily trained model robustness, eps 0.03137254901960784:
Accuracy on images: 92.69
Testing adversarily trained model robustness, eps 0.06274509803921569:
Accuracy on images: 87.68
Testing adversarily trained model robustness, eps 0.12549019607843137:
Accuracy on images: 66.38
Testing adversarily FGSM, eps 0:
Accuracy on images: 95.87
Testing adversarily FGSM, eps 0.00392156862745098:
Accuracy on images: 95.83
Testing adversarily FGSM, eps 0.00784313725490196:
Accuracy on images: 95.76
Testing adversarily FGSM, eps 0.01568627450980392:
Accura